In [30]:
import mlflow

In [38]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
mlflow.set_tracking_uri("sqlite:///mlflow.db")

In [39]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.search_experiments()

[<Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/3', creation_time=1723987917104, experiment_id='3', last_update_time=1723987917104, lifecycle_stage='active', name='log-best-model', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/2', creation_time=1723987460942, experiment_id='2', last_update_time=1723987460942, lifecycle_stage='active', name='lgbm-best-model', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/1', creation_time=1723986821985, experiment_id='1', last_update_time=1723986821985, lifecycle_stage='active', name='compare-models', tags={}>,
 <Experiment: artifact_location='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/0', creation_time=1723986821840, experiment_id='0', last_update_time=1723986821840, lifecycle_stage='active', name='Default', tags={}>]

In [40]:
runs = client.search_runs(
    experiment_ids='3',
    filter_string="metrics.rmse_lgbm  < 0.885",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse_lgbm ASC"]
)

In [41]:
runs[0]

<Run: data=<RunData: metrics={'rmse_lgbm': 0.8812156810825201}, params={'model': 'lgbm',
 'test-data-path': '../data/london_weather.csv',
 'train-data-path': '../data/london_weather.csv'}, tags={'developer': 'hema',
 'mlflow.log-model.history': '[{"run_id": "4321b0600e9043959229d555b3b69099", '
                             '"artifact_path": "lgbm_reg", "utc_time_created": '
                             '"2024-08-18 13:31:59.444725", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": 

In [42]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse_lgbm']:.4f}")

run id: 4321b0600e9043959229d555b3b69099, rmse: 0.8812


In [43]:
client.get_run('4321b0600e9043959229d555b3b69099')

<Run: data=<RunData: metrics={'rmse_lgbm': 0.8812156810825201}, params={'model': 'lgbm',
 'test-data-path': '../data/london_weather.csv',
 'train-data-path': '../data/london_weather.csv'}, tags={'developer': 'hema',
 'mlflow.log-model.history': '[{"run_id": "4321b0600e9043959229d555b3b69099", '
                             '"artifact_path": "lgbm_reg", "utc_time_created": '
                             '"2024-08-18 13:31:59.444725", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.0.2", '
                             '"serialization_format": 

In [44]:
run_id = "4321b0600e9043959229d555b3b69099"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="london-temperature-lgbm-regressor")

Successfully registered model 'london-temperature-lgbm-regressor'.
Created version '1' of model 'london-temperature-lgbm-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1723990931547, current_stage='None', description=None, last_updated_timestamp=1723990931547, name='london-temperature-lgbm-regressor', run_id='4321b0600e9043959229d555b3b69099', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/3/4321b0600e9043959229d555b3b69099/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [45]:
model_name = "london-temperature-lgbm-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [46]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1723990931547, current_stage='Staging', description=None, last_updated_timestamp=1723990945411, name='london-temperature-lgbm-regressor', run_id='4321b0600e9043959229d555b3b69099', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/3/4321b0600e9043959229d555b3b69099/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [47]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1723990931547, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2024-08-18', last_updated_timestamp=1723990949800, name='london-temperature-lgbm-regressor', run_id='4321b0600e9043959229d555b3b69099', run_link=None, source='/home/ubuntu/mlops_temperature_prediction/experiment-tracking/mlruns/3/4321b0600e9043959229d555b3b69099/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>